Dropout statistics data analyis
=====
This notebook summarizes the efforts made in the analysis of statistical dropout data for the standard 4130cu. in. array with Bolt LLXT guns. Other arrays or dropout rules may easily be analyzed by saving this Jupyter Notebook with a new name and feed in the raw statistics (dan) files for the relavant source. Dropout rules are specified in this notebook under the Heading called Dropout Rule.

## Motivation
Dropout specs and modeling has been a topic for discussion for years:
The tolerance specs are sharp, resulting in a red or green box in a dropout matrix, but the matrix is based on a set of assumptions that are quite inaccurate.

<li>Source modeling code and calibration: old vs new matters</li>

<li>Temperature sensitivity</li>

<li>We are assuming a static and 100% correctly deployed source.</li>

<li>Dropouts are done on vertical farfields, even if relevant farfields are mostly non-vertical</li>

<li>Origin of specs (operational more than geophysical, “nice round numbers”).</li>

<li>Etc.</li>

Reshoots due to dropouts are costly. A revision is in line with a streamlining/cost-cutting philosophy
There is no guarantee that a reshoot results in improved data quality
Current practice of bespoke dropout matrix per project is error prone and resource demanding. And triggers discussion if new modeling should be used to allow for more dropped guns.

Shot-to-shot designature: Some possibility to correct for a range of bad shots.

Include perl script

In [ ]:
nucver = "280"
sourceconf = "4135"
guntype = "H"

# Importing modules for plotting

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Setting some parameters for reading the input


Below, the header of two dan-files (broadband and conventional/cross-correlation) are listed to confirm the modeling parameters used in Nucleus. First specify the path to the dan files for the dataset to analyze:

In [ ]:
#path = "./release_280/batch_4135/"
path = "./release_" + nucver + "/batch_" + sourceconfig+ "/"

In [ ]:
file = "4135H__040_2000_080_5.dan"
f = open(path + "bb_" + file)
lines = f.read().splitlines()
f.close()
for i, line in enumerate(lines):
    if i < 56:
        print(i, line)


In [ ]:
file = "4135H__040_2000_080_5.dan"
f = open(path + "cc_" + file)
lines = f.read().splitlines()
f.close()
for i, line in enumerate(lines):
    if i < 56:
        print(i, line)


Setting the numbers of lines to skip. This is for the 4130 array.

In [ ]:
#bb statistics
onegunskip = 43
twogunskip = 74
spareskip = 539
eof = 557

#conventional statistics
onegunskipcc = 55
twogunskipcc = 86
spareskipcc = 551
eofcc = 569

arrayvol = '4135H'
arraydepth = [4, 5, 6, 7, 8, 9]
subsep = [8, 10]
temp = [5, 10, 15, 20, 25]
prefix = ['bb', 'cc']

Setting file name

In [ ]:
filenames = []

for a in arraydepth:
	for s in subsep:
		for t in temp:
			if s < 10: 
				filenames.append(arrayvol + '__0' + str(a) + '0_2000_0' + str(s) + '0_' + str(t) + '.dan')
			else:
				filenames.append(arrayvol + '__0' + str(a) + '0_2000_' + str(s) + '0_' + str(t) + '.dan')

Just checking the file names:

In [ ]:
count = 0
for filename in filenames:
    if count < 10:
        print(filename)
    count += 1
print(len(filenames))

## Modeling Parameters
This report analyse statistical values of the following dropout scenarios:
<li>Temperatures: 5, 10, 15, 20, 25</li>
<li>Depths 4-9m; 1m increment</li>
<li>Subarray separation: 8 and 10m</li>
<li>Bolt and GII guns</li>
<li>Broadband (refl coeff zero) and conventional (refl coeff -1)</li>
Total: 123 360 combinations
<li>60 source/temperature combinations</li>
<li>2 modelings per source (broadband and conv)</li>
<li>2 gun types</li>
<li>All single and two gun dropouts with spare gun substitution</li>
<li>514 dropout combinations per source (nominal source always used as reference)</li>

The datasets are analyzed separately for Gun type to detect systematic differences.


Defining the column names:

In [ ]:
columns_one_gun_bb_raw = ['droparray1', 'dropgun1', 'gunvolume1', 'AvgdB', 'MaxdB', 'MaxPhase']
columns_two_gun_bb_raw = ['droparray1', 'dropgun1', 'droparray2', 'dropgun2', 'gunvolume1', 'gunvolume2', 'AvgdB', 'MaxdB', 'MaxPhase']
columns_spare_gun_bb_raw = ['droparray1', 'dropgun1', 'droparray2', 'dropgun2', 'gunvolume1', 'gunvolume2', 'AvgdB', 'MaxdB', 'MaxPhase']	
columns_one_gun_cc_raw = ['droparray1', 'dropgun1', 'gunvolume1', 'Peak', 'Peakch','PtoB','PtoBch', 'x-corr', 'AvgdB', 'MaxdB']
columns_two_gun_cc_raw = ['droparray1', 'dropgun1', 'droparray2', 'dropgun2', 'gunvolume1', 'gunvolume2', 'Peak', 'Peakch', 'PtoB', 'PtoBch', 'x-corr', 'AvgdB', 'MaxdB']
columns_spare_gun_cc_raw = ['droparray1', 'dropgun1', 'droparray2', 'dropgun2', 'gunvolume1', 'gunvolume2', 'Peak', 'Peakch', 'PtoB', 'PtoBch', 'x-corr', 'AvgdB', 'MaxdB']

Statistical data defined as panda dataframes:

In [ ]:
stat_one_gun_bb_raw = pd.DataFrame()
stat_two_gun_bb_raw = pd.DataFrame()
stat_spare_gun_bb_raw = pd.DataFrame()
stat_one_gun_cc_raw = pd.DataFrame()
stat_two_gun_cc_raw = pd.DataFrame()
stat_spare_gun_cc_raw = pd.DataFrame()

Reading data from files into dataframes:

In [ ]:

for filename in filenames:
    temp_stat_one_gun_bb_raw = pd.read_csv(path + 'bb_' + filename, names=columns_one_gun_bb_raw, skiprows=onegunskip, delim_whitespace=True, nrows=twogunskip-onegunskip)
    temp_stat_one_gun_bb_raw['filename'] = filename
    temp_stat_one_gun_bb_raw['Depth'] = int(filename[7:10])
    temp_stat_one_gun_bb_raw['Subsep'] = int(filename[16:19])
    temp_stat_one_gun_bb_raw['Temperature'] = int(filename[20:22].replace('.',''))
    stat_one_gun_bb_raw = stat_one_gun_bb_raw.append(temp_stat_one_gun_bb_raw)
    temp_stat_one_gun_bb_raw = []
    
    temp_stat_two_gun_bb_raw = pd.read_csv(path + 'bb_' + filename, names=columns_two_gun_bb_raw, skiprows=twogunskip, delim_whitespace=True, nrows=spareskip-twogunskip)
    temp_stat_two_gun_bb_raw['filename'] = filename
    temp_stat_two_gun_bb_raw['Depth'] = int(filename[7:10])
    temp_stat_two_gun_bb_raw['Subsep'] = int(filename[16:19])
    temp_stat_two_gun_bb_raw['Temperature'] = int(filename[20:22].replace('.',''))
    stat_two_gun_bb_raw = stat_two_gun_bb_raw.append(temp_stat_two_gun_bb_raw)
    temp_stat_two_gun_bb_raw = []
    
    temp_stat_spare_gun_bb_raw = pd.read_csv(path + 'bb_' + filename, names=columns_spare_gun_bb_raw, skiprows=spareskip, delim_whitespace=True, nrows=eof-spareskip)
    temp_stat_spare_gun_bb_raw['filename'] = filename
    temp_stat_spare_gun_bb_raw['Depth'] = int(filename[7:10])
    temp_stat_spare_gun_bb_raw['Subsep'] = int(filename[16:19])
    temp_stat_spare_gun_bb_raw['Temperature'] = int(filename[20:22].replace('.',''))
    stat_spare_gun_bb_raw = stat_spare_gun_bb_raw.append(temp_stat_spare_gun_bb_raw)
    temp_stat_spare_gun_bb_raw = []
    
    temp_stat_one_gun_cc_raw = pd.read_csv(path + 'cc_' + filename, names=columns_one_gun_cc_raw, skiprows=onegunskipcc, delim_whitespace=True, nrows=twogunskipcc-onegunskipcc)
    temp_stat_one_gun_cc_raw['filename'] = filename
    temp_stat_one_gun_cc_raw['Depth'] = int(filename[7:10])
    temp_stat_one_gun_cc_raw['Subsep'] = int(filename[16:19])
    temp_stat_one_gun_cc_raw['Temperature'] = int(filename[20:22].replace('.',''))
    stat_one_gun_cc_raw = stat_one_gun_cc_raw.append(temp_stat_one_gun_cc_raw)
    temp_stat_one_gun_cc_raw = []
    
    temp_stat_two_gun_cc_raw = pd.read_csv(path + 'cc_' + filename, names=columns_two_gun_cc_raw, skiprows=twogunskipcc, delim_whitespace=True, nrows=spareskipcc-twogunskipcc)
    temp_stat_two_gun_cc_raw['filename'] = filename
    temp_stat_two_gun_cc_raw['Depth'] = int(filename[7:10])
    temp_stat_two_gun_cc_raw['Subsep'] = int(filename[16:19])
    temp_stat_two_gun_cc_raw['Temperature'] = int(filename[20:22].replace('.',''))
    stat_two_gun_cc_raw = stat_two_gun_cc_raw.append(temp_stat_two_gun_cc_raw)
    temp_stat_two_gun_cc_raw = []
    
    temp_stat_spare_gun_cc_raw = pd.read_csv(path + 'cc_' + filename, names=columns_spare_gun_cc_raw, skiprows=spareskipcc, delim_whitespace=True, nrows=eofcc-spareskipcc)
    temp_stat_spare_gun_cc_raw['filename'] = filename
    temp_stat_spare_gun_cc_raw['Depth'] = int(filename[7:10])
    temp_stat_spare_gun_cc_raw['Subsep'] = int(filename[16:19])
    temp_stat_spare_gun_cc_raw['Temperature'] = int(filename[20:22].replace('.',''))
    stat_spare_gun_cc_raw = stat_spare_gun_cc_raw.append(temp_stat_spare_gun_cc_raw)
    temp_stat_spare_gun_cc_raw = []

The follwing is a lambda function that converts to string and adds zeros to a total length of 2 characters to the input x (which could be an int). Is later used to convert all gun numbers from int to string. (3 to "03", for example)

In [ ]:
add_zero = lambda x: str(int(x)).zfill(2)

Adding all the string representations for gun numbering in a dedicated set of columns. Is later used to make array_gun_number index with a logical order. 

In [ ]:
stat_one_gun_bb_raw['dropgun1_str'] = stat_one_gun_bb_raw['dropgun1'].apply(add_zero)
stat_two_gun_bb_raw['dropgun1_str'] = stat_two_gun_bb_raw['dropgun1'].apply(add_zero)
stat_two_gun_bb_raw['dropgun2_str'] = stat_two_gun_bb_raw['dropgun2'].apply(add_zero)
stat_spare_gun_bb_raw['dropgun1_str'] = stat_spare_gun_bb_raw['dropgun1'].apply(add_zero)
stat_spare_gun_bb_raw['dropgun2_str'] = stat_spare_gun_bb_raw['dropgun2'].apply(add_zero)
stat_one_gun_cc_raw['dropgun1_str'] = stat_one_gun_cc_raw['dropgun1'].apply(add_zero)
stat_two_gun_cc_raw['dropgun1_str'] = stat_two_gun_cc_raw['dropgun1'].apply(add_zero)
stat_two_gun_cc_raw['dropgun2_str'] = stat_two_gun_cc_raw['dropgun2'].apply(add_zero)
stat_spare_gun_cc_raw['dropgun1_str'] = stat_spare_gun_bb_raw['dropgun1'].apply(add_zero)
stat_spare_gun_cc_raw['dropgun2_str'] = stat_spare_gun_bb_raw['dropgun2'].apply(add_zero)

In [ ]:
stat_one_gun_cc_raw['Bubble'] = stat_one_gun_cc_raw.apply(lambda row: row.Peak / row.PtoB, axis=1)
stat_two_gun_cc_raw['Bubble'] = stat_two_gun_cc_raw.apply(lambda row: row.Peak / row.PtoB, axis=1)
stat_spare_gun_cc_raw['Bubble'] = stat_spare_gun_cc_raw.apply(lambda row: row.Peak / row.PtoB, axis=1)

Printing some statistics to check that dataframes are ok (Remove hash for what you want to print)

In [ ]:
#print('stat_one_gun_bb_raw: ')
#print(stat_one_gun_bb_raw.head())
#print(stat_one_gun_bb_raw.head())
#print(stat_one_gun_bb_raw.tail())
#print(stat_one_gun_cc_raw)
#print(stat_two_gun_bb_raw.info())
#print('stat_spare_gun')

#print(stat_one_gun_cc_raw.head())
print(stat_one_gun_cc_raw.tail())
#print('stat_two_gun_cc_raw: ')
#print(stat_two_gun_cc_raw.head())
#print(stat_two_gun_cc_raw.info())
# print('stat_spare_gun_cc_raw')
#print(stat_spare_gun_cc_raw.tail())
# print(stat_spare_gun_cc_raw.info())
#print(len(stat_one_gun_bb_raw), len(stat_two_gun_bb_raw), len(stat_spare_gun_bb_raw))
#print(len(stat_one_gun_cc_raw), len(stat_two_gun_cc_raw), len(stat_spare_gun_cc_raw))

Next step is to copy dataframes to identical copy with "all" postfix. Then later, the mastermatrix filter will be applied to the original dataframes. The "all" dataframes will contain the unfiltered data material.

In [ ]:
stat_one_gun_bb_legal = stat_one_gun_bb_raw
stat_two_gun_bb_legal = stat_two_gun_bb_raw
stat_spare_gun_bb_legal = stat_spare_gun_bb_raw
stat_one_gun_cc_legal = stat_one_gun_cc_raw
stat_two_gun_cc_legal = stat_two_gun_cc_raw
stat_spare_gun_cc_legal = stat_spare_gun_cc_raw

## Dropout Rule
Master Matrix suggestion:
<li>Illegal dropouts are defined by operationally suitable gun combinations and not by “geophysical” specs:</li>

<li>All substitutions allowed</li>

<li>Same rule applied regardless of depth, gun type, subarray separation, temperature.</li>

<li>We probably have to live with some outliers (keep shooting in some of the red boxes). This notebook investigates the impact on a statistical level.</li>

The goal for the new dropout rule is:
<b>“No single gun dropout is a show- stopper. All failing big guns in clusters may be substituted”</b>

### Master Matrix definition and filtering of dataset
Here comes the master matrix definition and filtering:

In [ ]:
illegal_one_gun = [[1, 1], 
                   [1, 2],
                   [1, 10], #Single 100 may be substituted
				   [1, 13],
				   [1, 14],
                   [2, 1], # 100c may be substituted
                   [2, 2], # 100C may be substituted
                   [2, 9], # Single 100 may be substituted
                   [2, 11],
                   [2, 12],
                   [2, 13], # Singel 250 may be substituted
				   [3, 1],
				   [3, 2], 
                   [3, 3], # single 150 may be substituted
                   [3, 11], 
				   [3, 12],
				   [3, 13],
				   [3, 14]]
for i in range(len(illegal_one_gun)):
	stat_one_gun_bb_legal = stat_one_gun_bb_legal.drop(stat_one_gun_bb_legal[(stat_one_gun_bb_legal.droparray1 == illegal_one_gun[i][0]) & (stat_one_gun_bb_legal.dropgun1 == illegal_one_gun[i][1])].index, axis=1)
	stat_one_gun_cc_legal = stat_one_gun_cc_legal.drop(stat_one_gun_cc_legal[(stat_one_gun_cc_legal.droparray1 == illegal_one_gun[i][0]) & (stat_one_gun_cc_legal.dropgun1 == illegal_one_gun[i][1])].index, axis=1)

#Below it is assumed that any two gun combination with illegal one gun is also illegal.

for i in range(len(illegal_one_gun)):
	stat_two_gun_bb_legal = stat_two_gun_bb_legal.drop(stat_two_gun_bb_legal[(stat_two_gun_bb_legal.droparray1 == illegal_one_gun[i][0]) & (stat_two_gun_bb_legal.dropgun1 == illegal_one_gun[i][1]) | (stat_two_gun_bb_legal.droparray2 == illegal_one_gun[i][0]) & (stat_two_gun_bb_legal.dropgun2 == illegal_one_gun[i][1])].index, axis=0)
	stat_two_gun_cc_legal = stat_two_gun_cc_legal.drop(stat_two_gun_cc_legal[(stat_two_gun_cc_legal.droparray1 == illegal_one_gun[i][0]) & (stat_two_gun_cc_legal.dropgun1 == illegal_one_gun[i][1]) | (stat_two_gun_cc_legal.droparray2 == illegal_one_gun[i][0]) & (stat_two_gun_cc_legal.dropgun2 == illegal_one_gun[i][1])].index, axis=0)

In [ ]:
print("Number of combinations for all(raw) combinations (one gun, two gun and spare)")
print(len(stat_one_gun_bb_raw), len(stat_two_gun_bb_raw), len(stat_spare_gun_bb_raw))
print(len(stat_one_gun_cc_raw), len(stat_two_gun_cc_raw), len(stat_spare_gun_cc_raw))
print("Number of combinations for legal combinations (one gun, two gun and spare)")
print(len(stat_one_gun_bb_legal), len(stat_two_gun_bb_legal), len(stat_spare_gun_bb_legal))
print(len(stat_one_gun_cc_legal), len(stat_two_gun_cc_legal), len(stat_spare_gun_cc_legal))

Here comes the functions that do the matplotlib plots:

In [ ]:
def figplot(figno, dataset_bb_all, dataset_bb, dataset_cc_all, dataset_cc, figtitle, filename):
	plt.figure(figno, figsize=(16, 12))
	plt.suptitle(figtitle, fontsize=16)
	plt.subplot(321)
	plt.hist(dataset_bb_all['MaxPhase'], bins=40, range=(0, 40), alpha=0.4, label='all')
	#plt.hist(dataset_bb['MaxPhase'], bins=40, range=(0, 40), alpha=0.4, label='legal')
	plt.axvline(x=20)
	plt.legend()
	plt.title('MaxPhase')

	plt.subplot(323)
	plt.hist(dataset_bb_all['MaxdB'], bins=30, range=(0, 6), alpha=0.4, label='all')
	#plt.hist(dataset_bb['MaxdB'], bins=30, range=(0, 6), alpha=0.4, label='legal')
	plt.axvline(x=3)
	plt.legend()
	plt.title('MaxdB')

	plt.subplot(325)
	plt.hist(dataset_bb_all['AvgdB'], bins=40, range=(0, 1), alpha=0.4, label='all')
	#plt.hist(dataset_bb['AvgdB'], bins=40, range=(0, 1), alpha=0.4, label='legal')
	plt.axvline(x=0.85)
	plt.legend()
	plt.title('AvgdB')

	plt.subplot(322)
	plt.hist(dataset_cc_all['x-corr'], bins=40, range=(0.99, 1), alpha=0.4, label='all')
	#plt.hist(dataset_cc['x-corr'], bins=40, range=(0.99, 1), alpha=0.4, label='legal')
	plt.axvline(x=0.998)
	plt.legend()
	plt.title('X-corr')

	plt.subplot(324)
	plt.hist(dataset_cc_all['Peakch'], bins=24, range=(-10, 2), alpha=0.4, label='all')
	#plt.hist(dataset_cc['Peakch'], bins=24, range=(-10, 2), alpha=0.4, label='legal')
	plt.axvline(x=-10)
	plt.legend()
	plt.title('Peakch %')

	plt.subplot(326)
	plt.hist(dataset_cc_all['PtoBch'], bins=25, range=(-40, 10), alpha=0.4, label='all')
	#plt.hist(dataset_cc['PtoBch'], bins=25, range=(-40, 10), alpha=0.4, label='legal')
	plt.axvline(x=-10)
	plt.axvline(x=-15)
	plt.legend()
	plt.title('PtoBch %')
	plt.savefig(filename)
	plt.show()


First plot is of all two gun dropouts

In [ ]:
def figplot_peak_bubble(figno, dataset_cc_all, dataset_cc, figtitle, filename):
	plt.figure(figno, figsize=(16, 12))
	plt.suptitle(figtitle, fontsize=16)
	plt.subplot(221)
	plt.hist(dataset_cc_all['Peak'], bins=40, range=(50, 90), alpha=0.4, label='all')
	#plt.hist(dataset_bb['MaxPhase'], bins=40, range=(0, 40), alpha=0.4, label='legal')
	plt.legend()
	plt.title('Peak')

	plt.subplot(222)
	plt.hist(dataset_cc_all['Bubble'], bins=30, range=(0, 6), alpha=0.4, label='all')
	#plt.hist(dataset_bb['MaxdB'], bins=30, range=(0, 6), alpha=0.4, label='legal')
	plt.legend()
	plt.title('Bubble')

	plt.subplot(223)
	plt.hist(dataset_cc_all['Peakch'], bins=24, range=(-10, 2), alpha=0.4, label='all')
	#plt.hist(dataset_bb['AvgdB'], bins=40, range=(0, 1), alpha=0.4, label='legal')
	plt.axvline(x=-10)
	plt.legend()
	plt.title('Peakch')

	plt.subplot(224)
	plt.hist(dataset_cc_all['PtoBch'], bins=25, range=(-40, 20), alpha=0.4, label='all')
	#plt.hist(dataset_cc['x-corr'], bins=40, range=(0.99, 1), alpha=0.4, label='legal')
	plt.axvline(x=-10)
	plt.legend()
	plt.title('PtoBch')

In [ ]:
figplot_peak_bubble(2, stat_two_gun_cc_raw, stat_two_gun_cc_legal, 'Two Gun dropout statistics 4135H 2.8.0 k=2.0', '4135H_2gun_280_pb.png')
        

In [ ]:
figplot(1, stat_two_gun_bb_raw, stat_two_gun_bb_legal, stat_two_gun_cc_raw, stat_two_gun_cc_legal, 'Two Gun dropout statistics 4135H 2.8.0 k=2.0', '4135H_2gun_280.png' )

#### Above plot comments
Typical tolerance specs are marked with vertical line for reference only.
Note that orange dataset (master matrix “passed”)is a sub-selection of blue dataset (partly transparent makes the orange slightly brown)
We observe that the master matrix drop out rule gets rid of most “bad” combinations, according to the current specs, but also some good ones that would be within the current specs.
The number of legal combinations that are above current specs are few and quite marginal.

Second plot is for single gun dropouts:

In [ ]:
figplot(2, stat_one_gun_bb_raw, stat_one_gun_bb_legal, stat_one_gun_cc_raw, stat_one_gun_cc_legal, 'One Gun dropout statistics 4135H 2.8.0 k=2.0', '4135H_1gun_280.png')

#### Above plot comments
Single gun dropouts are a lot more likely than two gun dropouts.
The Master Matrix rule is rejecting practically all bad combinations, according to traditional specs
Note: Spares not enabled

In [ ]:
figplot(3, stat_spare_gun_bb_raw, stat_spare_gun_bb_legal, stat_spare_gun_cc_raw, stat_spare_gun_cc_legal, 'Spare Gun dropout statistics 4135H 2.8.0 k=2.0', '4135H_spare_280.png')

#### Above plot comments
All spare substitutions allowed, so light blue bars are completely obscures by darker blue bars.
We observe some substitutions that for some parameter combinations could be problematic.

For more in depth analysis and fancier plots (using Pyhton Seaborn module), we join the datasets. First, we'll strip off some columns before the join and add columns for Scenario (which can have the following values: "One gun dropoout", "Two gun dropout" and "Spare substitution") and for Status (which can have the following values: "All" or "Legal"). At the end, all the data are concatenated to a dataset (panda dataframe) called comb_data_bb (bb reflecting that this is the broadband dataset)

In [ ]:
strip_one_gun_bb_all = stat_one_gun_bb_raw.drop(columns=['filename'])
strip_one_gun_bb_all['Scenario'] = "One gun dropout"
strip_one_gun_bb_all['Status'] = "All"
strip_one_gun_bb_legal = stat_one_gun_bb_legal.drop(columns=['filename'])
strip_one_gun_bb_legal['Scenario'] = "One gun dropout"
strip_one_gun_bb_legal['Status'] = "Legal"
strip_two_gun_bb_all = stat_two_gun_bb_raw.drop(columns=['filename'])
strip_two_gun_bb_all['Scenario'] = "Two gun dropout"
strip_two_gun_bb_all['Status'] = "All"
strip_two_gun_bb_legal = stat_two_gun_bb_legal.drop(columns=['filename'])
strip_two_gun_bb_legal['Scenario'] = "Two gun dropout"
strip_two_gun_bb_legal['Status'] = "Legal"
strip_spare_gun_bb_all = stat_spare_gun_bb_raw.drop(columns=['filename'])
strip_spare_gun_bb_all['Scenario'] = "Spare substitution"
strip_spare_gun_bb_all['Status'] = "All"
strip_spare_gun_bb_legal = stat_spare_gun_bb_legal.drop(columns=['filename'])
strip_spare_gun_bb_legal['Scenario'] = "Spare substitution"
strip_spare_gun_bb_legal['Status'] = "Legal"
comb_data_bb = pd.concat([strip_one_gun_bb_all, strip_one_gun_bb_legal, strip_two_gun_bb_all, strip_two_gun_bb_legal, strip_spare_gun_bb_all, strip_spare_gun_bb_legal], ignore_index=True)

In [ ]:
print(comb_data_bb.head())
print(comb_data_bb.tail())

Then we repeat the process for the cc data (cross-correlation)

In [ ]:
strip_one_gun_cc_all = stat_one_gun_cc_raw.drop(columns=['filename'])
strip_one_gun_cc_all['Scenario'] = "One gun dropout"
strip_one_gun_cc_all['Status'] = "All"
strip_one_gun_cc_legal = stat_one_gun_cc_legal.drop(columns=['filename'])
strip_one_gun_cc_legal['Scenario'] = "One gun dropout"
strip_one_gun_cc_legal['Status'] = "Legal"
strip_two_gun_cc_all = stat_two_gun_cc_raw.drop(columns=['filename'])
strip_two_gun_cc_all['Scenario'] = "Two gun dropout"
strip_two_gun_cc_all['Status'] = "All"
strip_two_gun_cc_legal = stat_two_gun_cc_legal.drop(columns=['filename'])
strip_two_gun_cc_legal['Scenario'] = "Two gun dropout"
strip_two_gun_cc_legal['Status'] = "Legal"
strip_spare_gun_cc_all = stat_spare_gun_cc_raw.drop(columns=['filename'])
strip_spare_gun_cc_all['Scenario'] = "Spare substitution"
strip_spare_gun_cc_all['Status'] = "All"
strip_spare_gun_cc_legal = stat_spare_gun_cc_legal.drop(columns=['filename'])
strip_spare_gun_cc_legal['Scenario'] = "Spare substitution"
strip_spare_gun_cc_legal['Status'] = "Legal"
comb_data_cc = pd.concat([strip_one_gun_cc_all, strip_one_gun_cc_legal, strip_two_gun_cc_all, strip_two_gun_cc_legal, strip_spare_gun_cc_all, strip_spare_gun_cc_legal], ignore_index=True)
print(comb_data_cc.head())
print(comb_data_cc.tail())

Just checking the size of the datasets (cc and bb datasets should have same length since both have same number of combinations)

In [ ]:
print("all datasets:")
print(len(strip_one_gun_bb_all), len(strip_two_gun_bb_all), len(strip_spare_gun_bb_all))
print(len(strip_one_gun_cc_all), len(strip_two_gun_cc_all), len(strip_spare_gun_cc_all))
print("legal dataset:")
print(len(strip_one_gun_bb_legal), len(strip_two_gun_bb_legal), len(strip_spare_gun_bb_legal))
print(len(strip_one_gun_cc_legal), len(strip_two_gun_cc_legal), len(strip_spare_gun_cc_legal))

Combining data using a join (left join, adding a suffix of "-cc" to identical column names, then dropping some redundant columns)

In [ ]:
comb_data = comb_data_bb.join(comb_data_cc, rsuffix="-cc").drop(columns=['Scenario-cc', 'Status-cc', 'Depth-cc', 'Subsep-cc', 'Temperature-cc', 'droparray1-cc', 'droparray2-cc', 'dropgun1-cc', 'dropgun2-cc', 'gunvolume1-cc', 'gunvolume2-cc', 'dropgun1_str-cc', 'dropgun2_str-cc'])
                                                                       
                                                                         

In [ ]:
print(comb_data.columns)

In [ ]:
#comb_data['drop_arr_gun_1'] = str(comb_data['droparray1']['dropgun1'])
#comb_data['drop_arr_gun_1'] = comb_data.droparray1.astype(str).str.cat(comb_data.dropgun1.astype(str), sep='_')
#comb_data['drop_arr_gun_2'] = comb_data.droparray2.astype(str).str.cat(comb_data.dropgun2.astype(str), sep='_')
comb_data['drop_arr_gun_1'] = comb_data.droparray1.astype(str).str.cat(comb_data.dropgun1_str, sep='_')
comb_data['drop_arr_gun_2'] = comb_data.droparray2.astype(str).str.cat(comb_data.dropgun2_str, sep='_')

Inspecting some data. Note that the columns AvgdB and MaxdB contain data from the broadband modeling, whereas AvgdB-cc and MaxdB-cc contains the conventional spec values.

In [ ]:
#print(len(comb_data))
#print(comb_data.tail())
print(comb_data['drop_arr_gun_1'].tail())

Split up the full dataframe in two separate dateaframes: one for legal combinations, one for all combinations.
The comb_data dataframe contain both "all" and "legal" combinations. Convenient when plotting with "hue"
The comb_data_legal contain only status = "legal"
The comb_data_all contain only status "All"

In [ ]:
comb_data_legal = comb_data.loc[comb_data['Status'] == "Legal"]
comb_data_all = comb_data.loc[comb_data['Status'] == "All"]

In [ ]:
print(len(comb_data))
print(len(comb_data_all))
print(len(comb_data_legal))

## Box Plot
The box plot (also known as a box-and-whisker plot) can tell us about how our dataset is distributed. It shows us the range of our dataset, gives us an idea about where a significant portion of our data lies, and whether or not any outliers are present.

How to interpret a box plot:

-The box represents the interquartile range (The difference between the upper and lower quartiles (Q3–Q1) of a data set. This range is used as a measure of data spread: spanning 50% of a data set and eliminating the influence of outliers (the highest and lowest quarters of a data set are removed).) 

-The line in the middle of the box is the median

-The end lines are the first and third quartiles (In order to determine the interquartile range, a data set is divided into four equal parts. Each separating value is called a quartile (the first, the second, etc.). The second quartile is the median)

-The diamonds show outliers


In [ ]:
sns.set_style("darkgrid")
sns.set_palette("pastel")
sns.set_context("paper")
sns.set_palette("Paired")
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
sns.boxplot(data=comb_data, x='Scenario', y='MaxPhase', hue='Status');

For reference the 20 degree maxphase limit is plotted as a red line.
Observations:

One gun dropout (which is far more likely than two gun dropouts): All legal (master matrix filtered) or 100% of combinations are within the 20 phase spec

Two gun dropouts: More than 75% of legal dropouts are within spec

Spare gun dropouts: More than 75% of legal dropouts are within spec

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(3, color='red')
sns.boxplot(data=comb_data, x='Scenario', y='MaxdB', hue='Status');

## Violin Plots
Violin plots provide more information than box plots because instead of mapping each individual data point, we get an estimation of the dataset from the KDE (Kernel Density Estimation).

There are two KDE plots that are symmetrical along the center line.
A white dot represents the median.
The thick black line in the center of each violin represents the interquartile range.
The lines that extend from the center are the confidence intervals - just on the box plots, a violin plot also displays the 95% confidence interval.

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
sns.violinplot(data=comb_data, x='Scenario', y='MaxPhase', hue='Status');

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(3, color='red')
sns.violinplot(data=comb_data, x='Scenario', y='MaxdB', hue='Status');

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(3, color='red')
sns.violinplot(data=comb_data, x='Scenario', y='MaxdB', hue='Status');

### Checking correlation between some statistical values
To answer questions like: If x-corr is bad, is always MaxPhase equally bad?
Does traditional broadband specs capture bad combinations also according to other specs?

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(0.998, color='red')
plt.axvline(20, color='red')
sns.regplot(x=comb_data_all["MaxPhase"], y=comb_data_all["x-corr"], fit_reg=False)
sns.regplot(x=comb_data_legal["MaxPhase"], y=comb_data_legal["x-corr"], fit_reg=False);

Comment to above figure:
The red spec lines form a 4-field matrix.

Many combinations are bad using both specs (lower right field).

But many combinations that fullfill the x-corr spec, fail the MaxPhase spec (upper right field), also quite significantly (>30 degrees).

Also many combinations that meet the MaxPhase spec fails the x-corr spec (lower left field).

Note that dark dots are still legal combinations according to the MasterMatrix. Light dots are all combinations.

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(0.998, color='red')
plt.axvline(3, color='red')
sns.regplot(x=comb_data_all["MaxdB"], y=comb_data_all["x-corr"], fit_reg=False)
sns.regplot(x=comb_data_legal["MaxdB"], y=comb_data_legal["x-corr"], fit_reg=False);

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
plt.axvline(3, color='red')
sns.regplot(x=comb_data_all["MaxdB"], y=comb_data_all["MaxPhase"], fit_reg=False)
sns.regplot(x=comb_data_legal["MaxdB"], y=comb_data_legal["MaxPhase"], fit_reg=False);

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(3, color='red')
plt.axvline(3, color='red')
sns.regplot(x=comb_data_all["MaxdB"], y=abs(comb_data_all["MaxdB-cc"]), fit_reg=False)
sns.regplot(x=comb_data_legal["MaxdB"], y=abs(comb_data_legal["MaxdB-cc"]), fit_reg=False);

Note: abs(MaxdB-cc) correlates with MaxdB (check why this is different between bb and conventional stats)

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(1.2, color='red')
plt.axvline(0.85, color='red')
sns.regplot(x=comb_data_all["AvgdB"], y=comb_data_all["AvgdB-cc"], fit_reg=False)
sns.regplot(x=comb_data_legal["AvgdB"], y=comb_data_legal["AvgdB-cc"], fit_reg=False);

## What about traditional sets of specs?
In previous section, we looked at the dependencies between single specs.
So what about traditional sets of specs?

Broadband MC specs (5-200Hz, refl coeff = 0):

<li>AvgdB: 0.85</li>
<li>MaxdB: 3.0</li>
<li>MaxPhase: 20</li>

Conventional (old MC) specs (10-70Hz, refl coeff = -1)

<li>x-corr: 0.998</li>
<li>AvgdB: 1.2</li>
<li>MaxdB: 3.0</li>

Let us apply this rule as filters to the "All" datasets

In [ ]:
comb_data_bb_legal = comb_data[(comb_data['AvgdB'] < 0.85) & (comb_data['MaxdB'] < 3) & (comb_data['MaxPhase'] < 20)]




In [ ]:
print(len(comb_data))
print(len(comb_data_bb_legal))

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
plt.axvline(3, color='red')
sns.regplot(x=comb_data_all["MaxdB"], y=comb_data_all["MaxPhase"], fit_reg=False)
sns.regplot(x=comb_data_bb_legal["MaxdB"], y=comb_data_bb_legal["MaxPhase"], fit_reg=False);

In [ ]:
comb_data_cc_legal = comb_data[(comb_data['AvgdB-cc'] < 1.2) & (comb_data['MaxdB-cc'] < 3) & (comb_data['x-corr'] > 0.998)]

In [ ]:
print(len(comb_data))
print(len(comb_data_cc_legal))

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
plt.axvline(3, color='red')
sns.regplot(x=comb_data_all["MaxdB"], y=comb_data_all["MaxPhase"], fit_reg=False)
sns.regplot(x=comb_data_cc_legal["MaxdB"], y=comb_data_cc_legal["MaxPhase"], fit_reg=False);

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(0.998, color='red')
plt.axvline(20, color='red')
sns.regplot(x=comb_data_all["MaxPhase"], y=comb_data_all["x-corr"], fit_reg=False)
sns.regplot(x=comb_data_bb_legal["MaxPhase"], y=comb_data_bb_legal["x-corr"], fit_reg=False);

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(0.998, color='red')
plt.axvline(20, color='red')
sns.regplot(x=comb_data_all["MaxPhase"], y=comb_data_all["x-corr"], fit_reg=False)
sns.regplot(x=comb_data_cc_legal["MaxPhase"], y=comb_data_cc_legal["x-corr"], fit_reg=False);

### Checking dependiencies with temperature and depth

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(0.998, color='red')
sns.boxplot(data=comb_data, x='Depth', y='x-corr', hue='Status');

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(0.998, color='red')
sns.stripplot(x=comb_data["Depth"], y=comb_data["x-corr"], hue=comb_data['Status']);

Not that this plot could be misleading wrt distribution (many dots on top of eachother), box plot shown previously mitigates this.

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(0.998, color='red')
sns.violinplot(data=comb_data, x='Depth', y='x-corr', hue='Status');

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(0.998, color='red')
sns.boxplot(data=comb_data, x='Temperature', y='x-corr', hue='Status');

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(0.998, color='red')
sns.stripplot(x=comb_data["Temperature"], y=comb_data["x-corr"], hue=comb_data['Status']);

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
sns.violinplot(data=comb_data, x='Temperature', y='MaxPhase', hue='Status');

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
sns.stripplot(data=comb_data, x='Depth', y='MaxPhase', hue='Status');

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
sns.boxplot(data=comb_data, x='Depth', y='MaxPhase', hue='Status');

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
sns.violinplot(data=comb_data, x='Depth', y='MaxPhase', hue='Status');

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
sns.violinplot(data=comb_data, x='Subsep', y='MaxPhase', hue='Status');

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
sns.boxplot(data=comb_data, x='Depth', y='MaxPhase', hue='Status');

In [ ]:
print(comb_data.columns)

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
sns.violinplot(data=comb_data[comb_data['Scenario'] == 'Spare substitution'], x='dropgun2', y='MaxPhase', hue='Status');

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
sns.boxplot(data=comb_data[comb_data['Scenario'] == 'Spare substitution'], x='drop_arr_gun_1', y='MaxPhase', hue='drop_arr_gun_2');

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(0.998, color='red')
sns.boxplot(data=comb_data[comb_data['Scenario'] == 'Spare substitution'], x='gunvolume1', y='x-corr');

In [ ]:
plt.figure(figsize=(12,8))
plt.axhline(20, color='red')
sns.boxplot(data=comb_data[comb_data['Scenario'] == 'Two gun dropout'], x='gunvolume1', y='MaxPhase', hue='Status');